# Проект: Анализ вакансии для аналитиков данных

Выполнила: Казанцева Дарья

Цель: Визуализировать информацию о рынке вакансий для аналитиков в Европе

## Парсинг данных из html-документа

### Установка и импорт необходимых библиотек

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
# библиотека для парсинга
pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=3f556cd5b43676669df8ae4bfc80d863f98062a6f588c720f41a093866ca78de
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
# библиотека для вычленения города и страны из строки
pip install locationtagger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 94.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=058a309583de71ac244a2298e

In [3]:
# библиотека для определения языка, на котором написана строка
pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=e4a55460d5040c6dbf6cd2686a58b7bfa4ef4a68fe58648d86dbe98a37147521
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [ ]:
# библиотека для получения координат по названию города
pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from geopy.geocoders import Nominatim
from functools import partial
from time import sleep

In [1]:
# from bs4 import BeautifulSoup
import csv
import pandas as pd
import datetime
import re
from datetime import datetime
from datetime import timedelta
# from langid import classify

In [5]:
import nltk
import spacy
 
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

import locationtagger

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Функция для обработки html-документа

В ячейке ниже оставляю закомментированный код, которым я пользовалась, чтобы увидеть структуру html-страницы и вычленять из нее нужные теги.

In [ ]:
# with open("/content/gdrive/MyDrive/Colab Notebooks/sheet2_lnk.html") as fp:
#     soup = BeautifulSoup(fp, 'lxml')

# print(soup.prettify())

Создадим функцию для обработки и применим ее к нашему документу, чтобы на выходе получить список словарей с необходимыми данными, который уже можно будет превратить в датафрейм

In [ ]:
def pars_row(row:str) -> dict:
  soup = BeautifulSoup(row, 'lxml')
  
  vacancy = {
      'title': soup.find('h2', class_='t-24 t-bold jobs-unified-top-card__job-title'),
      'location': soup.find('span', class_="jobs-unified-top-card__bullet"),
      'type_of_employment': soup.find('span', class_='jobs-unified-top-card__workplace-type'),
      'company': soup.find('span', class_='jobs-unified-top-card__company-name'),
      'size_of_company': soup.find('span', class_='jobs-company__inline-information'),
      'scope_of_company': soup.find('div', class_='t-14 mt5'),
      'hard_skills': soup.find_all('li'),
      'date': soup.find('span', class_='jobs-unified-top-card__posted-date'),
      'applicants': soup.find('span', class_='jobs-unified-top-card__applicant-count'),
  }

  for k, v in vacancy.items():
    if v and k != 'hard_skills':
      vacancy[k] = v.text.strip()

  if vacancy['applicants']:
    vacancy['applicants'] = int(vacancy['applicants'].split(' ')[0])
  
  if vacancy['hard_skills']:
    string_text = ''
    html_text = vacancy['hard_skills']
    for i in html_text:
      text = i.text.strip().lower()
      string_text = string_text + text
    vacancy['hard_skills'] = string_text

  return vacancy

Открываем csv-файл с нашей исходной таблице из задания и применяем к ней функцию

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/total_data.csv') as file:
    reader = csv.reader(file, delimiter=',')    
    next(reader)
    array_of_vacancy = []
    for row in reader:
      array_of_vacancy.append(pars_row(row[1]))

Наш список со словарями для создания сырой таблицы готов

In [ ]:
raw_data = pd.DataFrame(array_of_vacancy)
raw_data

,title,location,type_of_employment,company,size_of_company,scope_of_company,hard_skills,date,applicants
0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting\n \n 11-...,full-time · entry level11-50 employees · staff...,1 week ago,47.0
1,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,None,None,[],1 week ago,NaN
2,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,None,None,[],1 week ago,NaN
3,Data Analyst (Space & Planning),"South Molton, England, United Kingdom",On-site,Mole Valley Farmers,None,None,"analysing historic space & sales performance, ...",1 week ago,NaN
4,Data Analyst,"Lugano, Ticino, Switzerland",On-site,FORFIRM,None,None,manage business requests in the data warehouse...,2 weeks ago,NaN
...,...,...,...,...,...,...,...,...,...
993,Ingeniero de datos Power BI + Azure Datafactory,Spain,Remote,Apiux Tecnología,201-500 employees,Information Technology & Services\n \n ...,full-time · entry level201-500 employees · inf...,4 days ago,55.0
994,Stage | Data Analyst,"Vermezzo, Lombardy, Italy",Hybrid,CPM Italy,51-200 employees,Retail\n \n 51-200 employees\n...,full-time · entry level51-200 employees · reta...,2 weeks ago,NaN
995,Business-Analyst:in,Greater Munich Metropolitan Area,Hybrid,Computer Futures,"501-1,000 employees",Information Technology & Services\n \n ...,"full-time · associate501-1,000 employees · it ...",1 day ago,51.0
996,Junior Software Developer,"Milan, Lombardy, Italy",Hybrid,XCHANGING ITALY S.P.A.,201-500 employees,Computer Software\n \n 201-500...,full-time · entry level201-500 employees · sof...,4 days ago,125.0


## Предобработка данных

Запишем нашу сырую таблицу в формат csv и в дальнейшем уже будем работать напрямую с ней

In [ ]:
raw_data.to_csv (r'raw_data_0306.csv', index= False)

In [9]:
url='https://drive.google.com/file/d/10rT1E4nLEY4j41zSUuMzVxJkP2oaWlD8/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
raw_data_0306 = pd.read_csv(url)

In [10]:
raw_data_0306

,title,location,type_of_employment,company,size_of_company,scope_of_company,hard_skills,date,applicants
0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting\n \n 11-...,full-time · entry level11-50 employees · staff...,1 week ago,47.0
1,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN
2,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN
3,Data Analyst (Space & Planning),"South Molton, England, United Kingdom",On-site,Mole Valley Farmers,NaN,NaN,"analysing historic space & sales performance, ...",1 week ago,NaN
4,Data Analyst,"Lugano, Ticino, Switzerland",On-site,FORFIRM,NaN,NaN,manage business requests in the data warehouse...,2 weeks ago,NaN
...,...,...,...,...,...,...,...,...,...
993,Ingeniero de datos Power BI + Azure Datafactory,Spain,Remote,Apiux Tecnología,201-500 employees,Information Technology & Services\n \n ...,full-time · entry level201-500 employees · inf...,4 days ago,55.0
994,Stage | Data Analyst,"Vermezzo, Lombardy, Italy",Hybrid,CPM Italy,51-200 employees,Retail\n \n 51-200 employees\n...,full-time · entry level51-200 employees · reta...,2 weeks ago,NaN
995,Business-Analyst:in,Greater Munich Metropolitan Area,Hybrid,Computer Futures,"501-1,000 employees",Information Technology & Services\n \n ...,"full-time · associate501-1,000 employees · it ...",1 day ago,51.0
996,Junior Software Developer,"Milan, Lombardy, Italy",Hybrid,XCHANGING ITALY S.P.A.,201-500 employees,Computer Software\n \n 201-500...,full-time · entry level201-500 employees · sof...,4 days ago,125.0


### Проверка языков, используемых в описании вакансий

С помощью библиотеки langid попробуем определить, какие языка встречаются в названии вакансий

In [11]:
raw_data_0306['title_lang'] = raw_data_0306['title'].apply(lambda x: classify(x))

In [12]:
raw_data_0306['title_lang'] = raw_data_0306['title_lang'].apply(lambda x: x[0])

In [13]:
raw_data_0306.groupby('title_lang')['title_lang'].count().sort_values()

title_lang
br      1
no      1
mt      1
eo      1
lt      1
fi      1
hu      2
cs      3
et      3
pl      4
sv      7
da      7
sk     19
es     29
fr     33
nl     37
de     41
it     44
en    763
Name: title_lang, dtype: int64

В таблице почти 30% вакансий написаны не на английском языке, в основном на итальянском и немецком. Это возможно создаст некоторые трудности при поиске релевантных вакансий, но к сожалению мне не удалось найти хорошую библиотеку, которая правильно переводит текст вакансии. Те, которые я нашла, переводили неверно или не переводили совсем. Поэтому оставим как есть, этот раздел останется как возможность попрактиковаться с необычными библиотеками python. Столбец с языком удалим

In [14]:
raw_data_0306 = raw_data_0306.drop(columns='title_lang')

### Обработка локации, добавление столбцов с городом и страной

Воспользуемся библиотекой locationtagger, которая может определить, есть ли в строке упоминание страны, региона или города. С ее помощью мы сможем вытащить в отдельный столбец страну, если она будет в столбце 'location'. Создадим временный столбец с локацией, туда запишем четь исправленую локацию, так как в ходе работы выяснилось, что библиотека плохо ищет страны и города в тексте со словами Metropolitan Area|Greater |Area|Metropolitan Region

In [9]:
raw_data_0306['location_temp'] = raw_data_0306['location'].apply(lambda x: re.sub(r'(Metropolitan Area|Greater |Area|Metropolitan Region)', '', x))

In [12]:
def to_country(row):
  temp = locationtagger.find_locations(text = row).countries
  if len(temp) == 0:
    return  locationtagger.find_locations(text = row).other_countries
  else:
    return temp

In [13]:
raw_data_0306['country'] = raw_data_0306['location_temp'].apply(to_country)


0                 [Switzerland]
1              [United Kingdom]
2              [United Kingdom]
3              [United Kingdom]
4                 [Switzerland]
                 ...           
993                     [Spain]
994                     [Italy]
995    [Germany, United States]
996                     [Italy]
997                     [Spain]
Name: country, Length: 998, dtype: object

Столбец со странами почти заполнен. Есть несколько строк с пустыми списками. Заполним их заглушкой и выведем

In [54]:
raw_data_0306['country'] = raw_data_0306['country'].apply(lambda x: x[0] if len(x)>=1 else 'not_found')

In [58]:
raw_data_0306.query('country == "not_found"')['location_temp']

105            Wroclaw 
156               Iasi 
178            Wroclaw 
225    Banska Bystrica 
757          Eindhoven 
Name: location_temp, dtype: object

In [ ]:
raw_data_0306.query('country == "United States"')['location_temp']

По какой-то причине именно эти города библиотека не смогла распознать. Тут всего пять строк, поэтому можем заполнить их вручную. Еще несколько строк заполнились страной США, это некорректная работа библиотеки, заполним также вручную

In [63]:
raw_data_0306.loc[raw_data_0306['location_temp'] == 'Wroclaw ', 'country'] = "Poland"
raw_data_0306.query('location_temp == "Wroclaw "')['country']

105    Poland
178    Poland
Name: country, dtype: object

In [65]:
raw_data_0306.loc[raw_data_0306['location_temp'] == 'Iasi ', 'country'] = "Romania"
raw_data_0306.loc[raw_data_0306['location_temp'] == 'Banska Bystrica ', 'country'] = "Slovakia"
raw_data_0306.loc[raw_data_0306['location_temp'] == 'Eindhoven ', 'country'] = "Netherlands"
raw_data_0306.loc[raw_data_0306['country'] == 'United States', 'country'] = "Netherlands"
raw_data_0306.query('location_temp == "Iasi "')['country']

156    Romania
Name: country, dtype: object

In [67]:
raw_data_0306['country'].unique()

array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Poland', 'Hungary', 'Greece', 'Italy', 'Sweden',
       'Lithuania', 'Belgium', 'Luxembourg', 'Germany', 'Portugal',
       'Bulgaria', 'Spain', 'Czechia', 'Latvia', 'Estonia', 'Malta',
       'Austria', 'Romania', 'Slovakia', 'Norway', 'Denmark', 'Finland',
       'Monaco', 'Croatia'], dtype=object)

Теперь заполним города. Возьмем нулевой элемент списка локации, в большинстве случаев это город

In [70]:
raw_data_0306['city'] = raw_data_0306['location_temp'].apply(lambda x: x.split(',')[0].strip())

Но у нас есть строки, в которых указаны только страны, но не города. Сравним столбцы с временной локацией и городом и при равенстве заполним заглушкой. Нужно равенство с временным столбцом, так как есть страна Люксембург и город Люксембург, что не позволяет смотреть равество со столбцом страны

In [77]:
raw_data_0306.loc[raw_data_0306['city'] == raw_data_0306['location_temp'], 'city'] = 'not_found'

In [81]:
raw_data_0306['city']

0                           Basel
1                        Coventry
2                        Coventry
3                    South Molton
4                          Lugano
5                     Southampton
6                           Leeds
7                        Nuneaton
8                           Paris
9                       Cambridge
10                  West Midlands
11                       Schiphol
12                        Chester
13                      Cambridge
14                    Craven Arms
15                         Dublin
16                        Belfast
17                     Sunderland
18                     Montévrain
19                        Bristol
20                   South Molton
21                        Bristol
22                       Solihull
23                         Dublin
24                      Blackpool
25                         Cracow
26                          Dijon
27                     Alsónémedi
28                         Dublin
29            

In [82]:
raw_data_0306

,title,location,type_of_employment,company,size_of_company,scope_of_company,hard_skills,date,applicants,location_temp,country,city
0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting\n \n 11-...,full-time · entry level11-50 employees · staff...,1 week ago,47.0,"Basel, Basel, Switzerland",Switzerland,Basel
1,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN,"Coventry, England, United Kingdom",United Kingdom,Coventry
2,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN,"Coventry, England, United Kingdom",United Kingdom,Coventry
3,Data Analyst (Space & Planning),"South Molton, England, United Kingdom",On-site,Mole Valley Farmers,NaN,NaN,"analysing historic space & sales performance, ...",1 week ago,NaN,"South Molton, England, United Kingdom",United Kingdom,South Molton
4,Data Analyst,"Lugano, Ticino, Switzerland",On-site,FORFIRM,NaN,NaN,manage business requests in the data warehouse...,2 weeks ago,NaN,"Lugano, Ticino, Switzerland",Switzerland,Lugano
5,Data Analyst - Logistics,"Southampton, England, United Kingdom",On-site,"Butler, Bridge & May",NaN,NaN,[],6 days ago,NaN,"Southampton, England, United Kingdom",United Kingdom,Southampton
6,Data Analyst,"Leeds, England, United Kingdom",On-site,Maria Mallaband Care Group Ltd,NaN,NaN,maintain existing reporting environments and d...,3 weeks ago,NaN,"Leeds, England, United Kingdom",United Kingdom,Leeds
7,Data Analyst,"Nuneaton, England, United Kingdom",Hybrid,Kelly Group,NaN,NaN,gather and consolidate data from multiple sour...,2 days ago,NaN,"Nuneaton, England, United Kingdom",United Kingdom,Nuneaton
8,Data Analyst,"Paris, Île-de-France, France",On-site,eXalt,NaN,NaN,"full-time · entry level501-1,000 employees · i...",2 weeks ago,140.0,"Paris, Île-de-France, France",France,Paris
9,Data Analyst - Hybrid Working,"Cambridge, England, United Kingdom",On-site,Blue Arrow,NaN,NaN,analysing and interpreting data on demographic...,2 weeks ago,NaN,"Cambridge, England, United Kingdom",United Kingdom,Cambridge


Удалим вспомогательный столбец location_temp, он нам больше не пригодится

In [83]:
raw_data_0306 = raw_data_0306.drop(columns='location_temp')

,title,location,type_of_employment,company,size_of_company,scope_of_company,hard_skills,date,applicants,country,city
0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting\n \n 11-...,full-time · entry level11-50 employees · staff...,1 week ago,47.0,Switzerland,Basel
1,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN,United Kingdom,Coventry
2,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN,United Kingdom,Coventry
3,Data Analyst (Space & Planning),"South Molton, England, United Kingdom",On-site,Mole Valley Farmers,NaN,NaN,"analysing historic space & sales performance, ...",1 week ago,NaN,United Kingdom,South Molton
4,Data Analyst,"Lugano, Ticino, Switzerland",On-site,FORFIRM,NaN,NaN,manage business requests in the data warehouse...,2 weeks ago,NaN,Switzerland,Lugano
5,Data Analyst - Logistics,"Southampton, England, United Kingdom",On-site,"Butler, Bridge & May",NaN,NaN,[],6 days ago,NaN,United Kingdom,Southampton
6,Data Analyst,"Leeds, England, United Kingdom",On-site,Maria Mallaband Care Group Ltd,NaN,NaN,maintain existing reporting environments and d...,3 weeks ago,NaN,United Kingdom,Leeds
7,Data Analyst,"Nuneaton, England, United Kingdom",Hybrid,Kelly Group,NaN,NaN,gather and consolidate data from multiple sour...,2 days ago,NaN,United Kingdom,Nuneaton
8,Data Analyst,"Paris, Île-de-France, France",On-site,eXalt,NaN,NaN,"full-time · entry level501-1,000 employees · i...",2 weeks ago,140.0,France,Paris
9,Data Analyst - Hybrid Working,"Cambridge, England, United Kingdom",On-site,Blue Arrow,NaN,NaN,analysing and interpreting data on demographic...,2 weeks ago,NaN,United Kingdom,Cambridge


Перезапишем таблицу, добавив столбцы со страной и городом, так как библиотека работает достаточно долго

In [14]:
# pd.set_option('display.max_rows', None)

In [ ]:
raw_data_0306.to_csv (r'raw_data_0406.csv', index= False)

In [6]:
url='https://drive.google.com/file/d/1Zlyu3T-nIVQf9PuYTyXvmezEH7RAzkN0/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
raw_data_0406 = pd.read_csv(url)

In [16]:
# raw_data_0406 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/raw_data_0406.csv')
raw_data_0406

,title,location,type_of_employment,company,size_of_company,scope_of_company,hard_skills,date,applicants,country,city
0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting\n \n 11-...,full-time · entry level11-50 employees · staff...,1 week ago,47.0,Switzerland,Basel
1,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN,United Kingdom,Coventry
2,Data Analyst - Logistics,"Coventry, England, United Kingdom",On-site,Resolute Recruitment,NaN,NaN,[],1 week ago,NaN,United Kingdom,Coventry
3,Data Analyst (Space & Planning),"South Molton, England, United Kingdom",On-site,Mole Valley Farmers,NaN,NaN,"analysing historic space & sales performance, ...",1 week ago,NaN,United Kingdom,South Molton
4,Data Analyst,"Lugano, Ticino, Switzerland",On-site,FORFIRM,NaN,NaN,manage business requests in the data warehouse...,2 weeks ago,NaN,Switzerland,Lugano
...,...,...,...,...,...,...,...,...,...,...,...
993,Ingeniero de datos Power BI + Azure Datafactory,Spain,Remote,Apiux Tecnología,201-500 employees,Information Technology & Services\n \n ...,full-time · entry level201-500 employees · inf...,4 days ago,55.0,Spain,not_found
994,Stage | Data Analyst,"Vermezzo, Lombardy, Italy",Hybrid,CPM Italy,51-200 employees,Retail\n \n 51-200 employees\n...,full-time · entry level51-200 employees · reta...,2 weeks ago,NaN,Italy,Vermezzo
995,Business-Analyst:in,Greater Munich Metropolitan Area,Hybrid,Computer Futures,"501-1,000 employees",Information Technology & Services\n \n ...,"full-time · associate501-1,000 employees · it ...",1 day ago,51.0,Germany,Munich
996,Junior Software Developer,"Milan, Lombardy, Italy",Hybrid,XCHANGING ITALY S.P.A.,201-500 employees,Computer Software\n \n 201-500...,full-time · entry level201-500 employees · sof...,4 days ago,125.0,Italy,Milan


### Сфера деятельности компаний

Откорректируем данные в столбце со сферой деятельности, вытащив из строки необходимые нам данные

In [7]:
raw_data_0406['scope_of_company']

0      Staffing & Recruiting\n        \n          11-...
1                                                    NaN
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
993    Information Technology & Services\n        \n ...
994    Retail\n        \n          51-200 employees\n...
995    Information Technology & Services\n        \n ...
996    Computer Software\n        \n          201-500...
997    Information Technology & Services\n        \n ...
Name: scope_of_company, Length: 998, dtype: object

Заполним пропуски заглушкой, чтобы можно было применить функцию к столбцу

In [8]:
raw_data_0406['scope_of_company'] = raw_data_0406['scope_of_company'].fillna('not_found')

In [9]:
raw_data_0406['scope_of_company'] = raw_data_0406['scope_of_company'].apply(lambda x: re.search(r'.{1,}', x).group(0))

In [10]:
raw_data_0406['scope_of_company']

0                  Staffing & Recruiting
1                              not_found
2                              not_found
3                              not_found
4                              not_found
                     ...                
993    Information Technology & Services
994                               Retail
995    Information Technology & Services
996                    Computer Software
997    Information Technology & Services
Name: scope_of_company, Length: 998, dtype: object

### Дата публикации вакансии

In [11]:
raw_data_0406['date'].unique()

array(['1 week ago', '2 weeks ago', '6 days ago', '3 weeks ago',
       '2 days ago', '1 day ago', '4 days ago', '4 weeks ago',
       '3 days ago', '5 days ago', '12 minutes ago', '29 minutes ago',
       '5 hours ago', '8 hours ago', '6 hours ago', '9 hours ago',
       '11 hours ago', '12 hours ago', '7 hours ago', '10 hours ago'],
      dtype=object)

Создадим функцию для обработки даты

In [12]:
def correct_date(row):
  start_date = datetime(2023, 5, 23)
  new_row = row.split()
  num = int(new_row[0])
  if 'weeks' or 'week' in new_row:
    result_date = start_date - timedelta(weeks=num)
  elif 'days' or 'day' in new_row:
    result_date = start_date - timedelta(days=num)
  else:
    result_date = start_date
  return result_date

In [13]:
raw_data_0406['date'] = raw_data_0406['date'].apply(correct_date)

In [14]:
raw_data_0406['date']

0     2023-05-16
1     2023-05-16
2     2023-05-16
3     2023-05-16
4     2023-05-09
         ...    
993   2023-04-25
994   2023-05-09
995   2023-05-16
996   2023-04-25
997   2023-05-09
Name: date, Length: 998, dtype: datetime64[ns]

### Обработка дубликатов

In [15]:
raw_data_0406[raw_data_0406.duplicated()].count()

title                 112
location              112
type_of_employment    111
company               112
size_of_company       111
scope_of_company      112
hard_skills           112
date                  112
applicants            111
country               112
city                  112
dtype: int64

У нас получилось 112 дубликатов, полностью идентичные строки, избавимся от них

In [40]:
raw_data_0406 = raw_data_0406.drop_duplicates()

TypeError: ignored

In [43]:
raw_data_0406.reset_index(drop=True, inplace=True)

In [17]:
raw_data_0406.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 886 entries, 0 to 997
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               886 non-null    object        
 1   location            886 non-null    object        
 2   type_of_employment  819 non-null    object        
 3   company             884 non-null    object        
 4   size_of_company     853 non-null    object        
 5   scope_of_company    886 non-null    object        
 6   hard_skills         886 non-null    object        
 7   date                886 non-null    datetime64[ns]
 8   applicants          727 non-null    float64       
 9   country             886 non-null    object        
 10  city                886 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 83.1+ KB


### Обработка хард-скиллов

In [45]:
skills = [
    'a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'ai', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', ' r;', ' r,', ' r.' 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin'
]

Объявим функцию для поиска хард-скиллов из списка выше в столбце

In [46]:
def found_hard_skills(row):
  hard_skills = []
  for item in skills:
    if item in row:
      hard_skills.append(item)
  return hard_skills

In [47]:
raw_data_0406['hard_skills'] = raw_data_0406['hard_skills'].apply(found_hard_skills)
raw_data_0406['hard_skills']

0      [ai, conda, data mining, excel, sap, sas, segm...
1                                                     []
2                                            [ai, excel]
3      [ai, aws, data warehouse, etl, gcp, oracle, or...
4                                                     []
                             ...                        
881                                              [azure]
882                              [ai, excel, powerpoint]
883                                       [ai, api, sql]
884                  [ai, docker, git, java, nosql, sql]
885                                           [api, sap]
Name: hard_skills, Length: 886, dtype: object

### Удаление нерелевантных вакансий

Попробуем с помощью функции отфильтровать релевантные вакансии, воспользовавшись поиском по подстроке. Так как вакансии есть не только на английском языке, полностью название вакансии не подойдет, но как будто во всех языках можно найти дата и начало слова аналитик. Зададим условие, что аналитик и дата встречаются одновременно, а BI встречаются вместе с аналитиком.

In [48]:
def relev(row):
  stroka_1 = 'anal'
  stroka_2 = 'data'
  stroka_3 = 'bi'
  row_new = row.lower()
  if (stroka_1 in row_new and stroka_2 in row_new) or (stroka_1 in row_new and stroka_3 in row_new):
    return 'relevant'
  else:
    return 'not relevant'

In [49]:
raw_data_0406['relevance'] = raw_data_0406['title'].apply(relev)

In [50]:
raw_data_0406['relevance'].value_counts()

relevant        478
not relevant    408
Name: relevance, dtype: int64

У нас предварительно получилось всего 478 релевантных заданию вакансий

При беглом просмотре видим, что в целом все в порядке, есть дата аналитики и аналитики BI

### Добавление координат для построения карты в дата ленс

In [53]:
raw_data_0406 = raw_data_0406.reset_index().explode('hard_skills', ignore_index=True)

In [56]:
raw_data_0406.to_csv (r'final_data_1006.csv', index= False)

In [4]:
url='https://drive.google.com/file/d/194itEQA7Vh2oJDWfmeKb8PqXZac7WNRT/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
final_data = pd.read_csv(url)

Финальная таблица для визуализации почти готова, осталось добавить координаты

In [7]:
final_data

,index,title,location,type_of_employment,company,size_of_company,scope_of_company,hard_skills,date,applicants,country,city,relevance
0,0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,ai,2023-05-16,47.0,Switzerland,Basel,relevant
1,0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,conda,2023-05-16,47.0,Switzerland,Basel,relevant
2,0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,data mining,2023-05-16,47.0,Switzerland,Basel,relevant
3,0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,excel,2023-05-16,47.0,Switzerland,Basel,relevant
4,0,Data Analyst,"Basel, Basel, Switzerland",On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,sap,2023-05-16,47.0,Switzerland,Basel,relevant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,884,Junior Software Developer,"Milan, Lombardy, Italy",Hybrid,XCHANGING ITALY S.P.A.,201-500 employees,Computer Software,java,2023-04-25,125.0,Italy,Milan,not relevant
4712,884,Junior Software Developer,"Milan, Lombardy, Italy",Hybrid,XCHANGING ITALY S.P.A.,201-500 employees,Computer Software,nosql,2023-04-25,125.0,Italy,Milan,not relevant
4713,884,Junior Software Developer,"Milan, Lombardy, Italy",Hybrid,XCHANGING ITALY S.P.A.,201-500 employees,Computer Software,sql,2023-04-25,125.0,Italy,Milan,not relevant
4714,885,Analista de datos BI,"Valencia, Valencian Community, Spain",Hybrid,Infoport,51-200 employees,Information Technology & Services,api,2023-05-09,198.0,Spain,Valencia,relevant


In [ ]:
geolocator = Nominatim(user_agent="darya")

In [29]:
geocode = partial(geolocator.geocode, language="en")
print(geocode("london"))

London, Greater London, England, United Kingdom


Создадим список со всеми уникальными городами нашего датасета

In [35]:
cities = final_data['city'].unique()
cities

array(['Basel', 'Coventry', 'South Molton', 'Lugano', 'Southampton',
       'Leeds', 'Nuneaton', 'Paris', 'Cambridge', 'West Midlands',
       'Schiphol', 'Chester', 'Craven Arms', 'Dublin', 'Belfast',
       'Sunderland', 'Montévrain', 'Bristol', 'Solihull', 'Blackpool',
       'Cracow', 'Dijon', 'Alsónémedi', 'Manchester',
       'Elliniko-Argyroupoli', 'not_found', 'Umeå', 'North Holland',
       'Vilnius', 'Durham', 'Oudenaarde', 'Milan', 'Stockholm County',
       'Luxembourg', 'Roubaix', 'Munich', 'Zaventem', 'Brussels',
       'West Malling', 'Lille', 'Egham', 'Karlstad', 'Madrid',
       'Barcelona', 'Oxford', 'Taibon', 'Epsom', 'Amsterdam', 'Spinea',
       'Palma de Mallorca', 'Brindisi', 'Boulogne-Billancourt',
       'Wolfsburg', 'Nantes', 'Derby', 'Lund', 'Garwolin', 'Stockholm',
       'Rome', 'Massy', 'Prague', 'Middlesbrough', 'Viladecans', 'Warsaw',
       'Eindhoven', 'Budapest', 'London', 'Wroclaw', 'Amersfoort',
       'Hamburg', 'The Hague', 'Chappes', 'Sintra', 'R

Ниже объявлена функция для создания словаря с названием города и его координатами. Словарь уже заполнен, чтобы несколько раз не запускать функцию, так как работает она долго

In [2]:
coordinates = {"Basel": [47.5581077, 7.5878261],
"Coventry": [52.4081812, -1.510477],
"South Molton": [51.0170696, -3.8330599],
"Lugano": [46.0038279, 8.9512134],
"Southampton": [50.9025349, -1.404189],
"Leeds": [53.7974185, -1.5437941],
"Nuneaton": [52.5246182, -1.4684488],
"Paris": [48.8534951, 2.3483915],
"Cambridge": [52.2055314, 0.1186637],
"West Midlands": [52.5050033, -1.964396123331272],
"Schiphol": [52.3080392, 4.7621975],
"Chester": [53.1908873, -2.8908955],
"Craven Arms": [52.439652, -2.8356405],
"Dublin": [53.3498006, -6.2602964],
"Belfast": [54.596391, -5.9301829],
"Sunderland": [54.9058512, -1.3828727],
"Montévrain": [48.8741286, 2.748344],
"Bristol": [51.4538022, -2.5972985],
"Solihull": [52.4130189, -1.7768935],
"Blackpool": [53.8179442, -3.0509812],
"Cracow": [-25.330132499999998, 150.29240746995595],
"Dijon": [47.3215806, 5.0414701],
"Alsónémedi": [47.3146713, 19.165704],
"Manchester": [53.4794892, -2.2451148],
"Elliniko-Argyroupoli": [37.8925913, 23.7471137],
"not_found": [41.1851881, -122.9240849],
"Umeå": [63.8256568, 20.2630745],
"North Holland": [52.7212825, 4.820665],
"Vilnius": [54.6870458, 25.2829111],
"Durham": [35.996653, -78.9018053],
"Oudenaarde": [50.8433705, 3.6041443],
"Milan": [45.4641943, 9.1896346],
"Stockholm County": [59.3954053, 18.665269528206764],
"Luxembourg": [49.8158683, 6.1296751],
"Roubaix": [50.6915893, 3.1741734],
"Munich": [48.1371079, 11.5753822],
"Zaventem": [50.8806207, 4.4730008],
"Brussels": [50.8465573, 4.351697],
"West Malling": [51.2954028, 0.409461],
"Lille": [50.6365654, 3.0635282],
"Egham": [51.4312585, -0.5473085],
"Karlstad": [59.3809146, 13.5027631],
"Madrid": [40.4167047, -3.7035825],
"Barcelona": [41.3828939, 2.1774322],
"Oxford": [51.7520131, -1.2578499],
"Taibon": [46.31385445, 11.9680616932694],
"Epsom": [51.3326098, -0.2678212],
"Amsterdam": [52.3730796, 4.8924534],
"Spinea": [45.4911604, 12.165018],
"Palma de Mallorca": [39.5695818, 2.6500745],
"Brindisi": [40.63591975, 17.688443357842537],
"Boulogne-Billancourt": [48.8356649, 2.240206],
"Wolfsburg": [52.4205588, 10.7861682],
"Nantes": [47.2186371, -1.5541362],
"Derby": [52.9212617, -1.4761491],
"Lund": [55.7029296, 13.1929449],
"Garwolin": [51.8958048, 21.621064749998972],
"Stockholm": [59.3251172, 18.0710935],
"Rome": [41.8933203, 12.4829321],
"Massy": [49.6902106, 1.399492],
"Prague": [50.0874654, 14.4212535],
"Middlesbrough": [54.5760419, -1.2344047],
"Viladecans": [41.3163083, 2.0156034],
"Warsaw": [52.2319581, 21.0067249],
"Eindhoven": [51.4392648, 5.478633],
"Budapest": [47.48138955, 19.14609412691246],
"London": [51.5073359, -0.12765],
"Wroclaw": [51.1089776, 17.0326689],
"Amersfoort": [52.1562499, 5.3896944],
"Hamburg": [53.550341, 10.000654],
"The Hague": [52.0799838, 4.3113461],
"Chappes": [45.8682621, 3.2226312],
"Sintra": [38.79846, -9.3881],
"Riga": [56.9493977, 24.1051846],
"Coimbra": [40.2111931, -8.4294632],
"Tartu": [58.3801207, 26.72245],
"Île-de-France": [48.6443057, 2.7537863],
"Issy-les-Moulineaux": [48.8250508, 2.273457],
"Hawthorn": [-37.8244246, 145.0317207],
"Lyon": [45.7578137, 4.8320114],
"Valletta": [35.8989818, 14.5136759],
"Stuttgart": [48.7784485, 9.1800132],
"Galway": [53.2744122, -9.0490601],
"Berlin": [52.5170365, 13.3888599],
"Olèrdola": [41.321421650000005, 1.7234463352877167],
"Neuilly-sur-Seine": [48.884683, 2.2695658],
"Wasserburg (Bodensee)": [47.575463150000004, 9.633586478277877],
"Gdynia": [54.5164982, 18.5402738],
"Bordeaux": [44.841225, -0.5800364],
"Rouen": [49.4404591, 1.0939658],
"Mechelen": [51.0281381, 4.4803453],
"Milton Keynes": [52.0406502, -0.7594092],
"Bari": [41.1257843, 16.8620293],
"Vienna": [48.2083537, 16.3725042],
"Newcastle upon Tyne": [54.9738474, -1.6131572],
"Cerdanyola del Vallès": [41.4910324, 2.1374969],
"Naples": [40.8358846, 14.2487679],
"Nova Milanese": [45.5893205, 9.2003524],
"Baierbrunn": [48.0204773, 11.4865462],
"Parma": [44.8013678, 10.3280833],
"Lindau": [47.550753, 9.6926624],
"Bergen op Zoom": [51.4944756, 4.2871541],
"Iasi": [47.1615416, 27.5837224],
"Gdańsk": [54.42880315, 18.798325653091226],
"Bodelshausen": [48.3939085, 8.9722782],
"Marlow": [54.1556493, 12.5725117],
"Brussels Region": [50.8465573, 4.351697],
"La Défense": [48.891008, 2.2412078],
"Lisbon": [38.7077507, -9.1365919],
"Porto": [41.1494512, -8.6107884],
"The Randstad": [52.9543128, -1.1539786],
"Neuss": [51.1981778, 6.6916476],
"Ghent": [51.0538286, 3.7250121],
"San Ġiljan": [35.9199225, 14.4877119],
"Cologne": [50.938361, 6.959974],
"Rijswijk": [52.0446265, 4.3188215],
"Tessenderlo": [51.0681628, 5.0893974],
"St.-Ouen": [50.03683, 2.1223737],
"Buckinghamshire": [51.840894, -0.8998073587480795],
"Bologna": [44.4938203, 11.3426327],
"Hasselt": [50.9303735, 5.3378043],
"Bath": [51.3813864, -2.3596963],
"Dresden": [51.0493286, 13.7381437],
"Wiesbaden": [50.0820384, 8.2416556],
"Munster": [52.307621600000004, -8.570897276292667],
"Utrecht": [52.080985600000005, 5.12768396945229],
"Mülheim an der Ruhr": [51.4272925, 6.8829192],
"Zwaagdijk-Oost": [52.7047349, 5.1336218],
"Heerlen": [50.8775239, 5.981506585454879],
"Hatfield": [51.7634675, -0.2258741],
"Harlestone": [52.2741524, -0.9656214],
"Nanterre": [48.8924273, 2.2071267],
"Hook of Holland": [51.9771615, 4.1314526],
"Banska Bystrica": [48.7384028, 19.1573494],
"Nuremberg": [49.453872, 11.077298],
"Oslo": [59.9133301, 10.7389701],
"Szczecin": [53.4301818, 14.5509623],
"Villes": [46.089198, 5.7851152],
"Doncaster": [53.5227681, -1.1335312],
"County Dublin": [53.4064939, -6.287060576851394],
"Villeneuve-d’Ascq": [50.6193174, 3.1314002],
"Toulouse": [43.6044622, 1.4442469],
"Meudon": [48.8126688, 2.2385432],
"Padua": [45.4077172, 11.8734455],
"Rotterdam": [51.9244424, 4.47775],
"Levallois-Perret": [48.892956, 2.2881683],
"Copenhagen": [55.6867243, 12.5700724],
"Espoo": [60.2047672, 24.6568435],
"Karlsruhe": [49.0068705, 8.4034195],
"Limburg": [51.2015196, 5.9046302],
"Schwäbisch Gmünd": [48.7999036, 9.7977584],
"Sundsvall": [62.3907552, 17.3071024],
"Neckarsulm": [49.191133, 9.2248391],
"Düsseldorf": [51.2254018, 6.7763137],
"Gillingham": [51.3871701, 0.5461611],
"Bucharest": [44.4361414, 26.1027202],
"Emmen": [52.75286615, 6.952513466208563],
"Guissona": [41.785242, 1.289032],
"Hoofddorp": [52.305554, 4.6926644],
"Weinheim": [49.5462349, 8.6717458],
"West Flanders": [51.04047465, 2.9994213387887116],
"Busto Arsizio": [45.6119057, 8.8520522],
"Lisboa": [38.7077507, -9.1365919],
"Jena": [50.9281717, 11.5879359],
"Worksop": [53.3042362, -1.1246168],
"Vilvoorde": [50.928711, 4.423126],
"Bergisch Gladbach": [50.9929303, 7.1277379],
"Drammen": [59.6964936, 10.175641950012762],
"Frankfurt": [50.1106444, 8.6820917],
"Athens": [37.9839412, 23.7283052],
"Zeist": [52.086531750000006, 5.249007957671885],
"Groningen": [53.2190652, 6.5680077],
"Louvain-la-Neuve": [50.6741689, 4.613790587187648],
"Mannheim": [49.4892913, 8.4673098],
"Łódź": [51.7687323, 19.4569911],
"St.-Denis": [48.935773, 2.3580232],
"Grenzach-Wyhlen": [47.5528548, 7.6737981],
"Essen": [51.4582235, 7.0158171],
"Leipzig": [51.3406321, 12.3747329],
"Iráklion": [35.33908, 25.1332843],
"Vaprio d'Adda": [45.5765762, 9.5291744],
"Gloucestershire": [51.8333313, -2.1666674],
"Frankfurt am Main": [50.1106444, 8.6820917],
"Ahlen": [51.7627633, 7.8909086],
"Sofia": [42.6977028, 23.3217359],
"Cesate": [45.5941861, 9.0747366],
"Binasco": [45.3319695, 9.0999868],
"Bellinzago Lombardo": [45.542075, 9.445951],
"Cernusco sul Naviglio": [45.5245578, 9.330925],
"Colturano": [45.37977, 9.337852],
"Truccazzano": [45.483874, 9.468553],
"Inzago": [45.540475, 9.482052],
"Pistoia": [43.9740957, 10.868708114044026],
"Canegrate": [45.5705379, 8.9267921],
"Lombardy": [45.5703694, 9.7732524],
"Coburg": [50.258112, 10.964463],
"Pau": [43.2957547, -0.3685668],
"San Zenone al Lambro": [45.327269, 9.355253],
"Cassina de' Pecchi": [45.5191887, 9.3619763],
"Monaco": [43.7323492, 7.4276832],
"Höchberg": [49.7837494, 9.8811173],
"Palermo": [38.1112268, 13.3524434],
"Treviso": [45.806691349999994, 12.206315763116372],
"Thessaloniki": [40.6403167, 22.9352716],
"Courbevoie": [48.8953328, 2.2561602],
"Doesburg": [52.018750499999996, 6.151558274308546],
"Tussenhausen": [48.1019964, 10.5613528],
"Arluno": [45.50507, 8.941541],
"Hückelhoven": [51.0552368, 6.2247322],
"Eisenach": [50.9747134, 10.3193565],
"Hereford": [52.0553813, -2.7151735],
"Erstein": [48.4227385, 7.6632641],
"Szombathely": [47.2291184, 16.6187625],
"Ronneby": [56.3, 15.283333],
"Elmshorn": [53.7532486, 9.6524559],
"Pioltello": [45.5010216, 9.3263354],
"Bremen": [53.0758196, 8.8071646],
"Lidköping": [58.5037196, 13.1576427],
"Ede": [52.07168255, 5.745510631034939],
"Vannes": [47.6586772, -2.7599079],
"Vittuone": [45.487969, 8.952442],
"Faenza": [44.2855555, 11.8832055],
"Tallinn": [59.4372155, 24.7453688],
"Farum": [55.8101232, 12.3702505],
"Ziano di Fiemme": [46.2865173, 11.5659762],
"Rethymno": [35.3676472, 24.4736079],
"Braine-l’Alleud": [50.69430665, 4.3547606874647],
"Vila Nova de Cerveira": [41.9403912, -8.7436068],
"Campodarsego": [45.502855, 11.9074958],
"Bonn": [50.735851, 7.10066],
"Hertfordshire": [51.840052299999996, -0.09785519224523963],
"Lelystad": [52.5150949, 5.4768915],
"Marsa": [42.8225242, 2.1551881],
"Elsbethen": [47.7592351, 13.0825458],
"Augusta": [33.4709714, -81.9748429],
"Ottignies-Louvain-la-Neuve": [50.6654216, 4.5674107],
"Fuschl am See": [47.7963895, 13.3027787],
"Solna": [59.361367, 18.0013693],
"Wrocław": [51.1089776, 17.0326689],
"Salzburg": [47.7981346, 13.0464806],
"Senago": [45.5761376, 9.1244464],
"Åre": [63.4010888, 13.0822241],
"Bilbao-Bilbo": [43.2537991, -2.9118748],
"Bois-Colombes": [48.9148269, 2.2674892],
"Vitoria-Gasteiz": [42.8465088, -2.6724025],
"Cusano Milanino": [45.5519897, 9.1856198],
"Dresano": [45.3729705, 9.3606419],
"Clichy": [48.9026, 2.30551],
"Douai": [50.3675677, 3.0804641],
"s-Hertogenbosch": [51.6889387, 5.303116],
"Kaiserslautern": [49.4432174, 7.7689951],
"Zoetermeer": [52.06228915, 4.487754536970652],
"Cagliari": [39.2171994, 9.113311],
"Moulineaux": [49.3418677, 0.9647213],
"Zagreb": [45.84264135, 15.962231476593626],
"Flemish Region": [51.096246199999996, 4.178629103169916],
"Zwevezele": [51.0361481, 3.2106957],
"Košice": [48.7172272, 21.2496774],
"Calvignasco": [45.325966, 9.027746],
"Telde": [27.9984538, -15.4166761],
"Ronda": [36.7421339, -5.1665916],
"Chevigny-Saint-Sauveur": [47.2977106, 5.1382218],
"Gessate": [45.5543877, 9.4351918],
"Arrasate / Mondragón": [43.0656783, -2.4900779],
"Rozzano": [45.3832366, 9.1541309],
"Vanzago": [45.5268256, 8.9949548],
"Esch-Sur-Alzette": [49.4959628, 5.9850306],
"Gattinara": [45.6142424, 8.3711382],
"Danderyd": [59.4043647, 18.0358505],
"Sesto San Giovanni": [45.5357218, 9.2376549],
"Turin": [45.0677551, 7.6824892],
"Savona": [44.2334238, 8.252572737021062],
"Cornaredo": [45.5016851, 9.025746],
"Torremolinos": [36.6242841, -4.4995448],
"Lucerne": [47.0505452, 8.3054682],
"Monza": [45.639544, 9.278869369225038],
"Carpi": [44.7835699, 10.8854523],
"County Cork": [51.917535900000004, -8.58597726870895],
"Poissy": [48.927439, 2.0428293],
"Kontich": [51.1353297, 4.4454784],
"Frosinone": [41.628546799999995, 13.57584977062367],
"Amstelveen": [52.3025582, 4.8467875],
"San Donato Milanese": [45.41807, 9.27165],
"Valladolid": [41.6521328, -4.728562],
"Suresnes": [48.8709053, 2.2255727],
"Laudio / Llodio": [43.1417601, -2.9622358],
"Holzwickede": [51.5007037, 7.6186433],
"s-Hertogenbosch": [51.6889387, 5.303116],
"Lacchiarella": [45.3218476, 9.137047],
"Sant'Elpidio a Mare": [43.2347405, 13.6878517],
"Terranuova Bracciolini": [43.551208, 11.5858271],
"Gennevilliers": [48.9254221, 2.2940122],
"Sparta": [37.08114525, 22.42486178272022],
"Brunswick": [52.2646577, 10.5236066],
"Corsico": [45.4307363, 9.108939],
"Aalborg": [57.0462626, 9.9215263],
"Florence": [43.7698712, 11.2555757],
"Chaucer": [53.4281287, -1.4857301],
"Dobříš": [49.7813749, 14.1704372],
"Heidelberg": [49.4093582, 8.694724],
"Liguria": [44.4777617, 8.7026296],
"Cusago": [45.4498179, 9.0363223],
"Oeiras": [38.7124971, -9.271300382407272],
"Leinfelden-Echterdingen": [48.6901796, 9.1525725],
"Timişoara": [45.7538355, 21.2257474],
"Edinburgh": [55.9533456, -3.1883749],
"Bertrange": [49.3133457, 6.193137],
"Piraeus": [37.9431594, 23.6470593],
"Arnhem": [51.984257, 5.9108573],
"Sliema": [35.9127609, 14.5020708],
"Aix-en-Provence": [43.5298424, 5.4474738],
"Gießen": [50.5862066, 8.6742306],
"Debrecen": [47.531399, 21.6259782],
"Bergamo": [45.756655699999996, 9.754219200862249],
"Ulm": [48.3974003, 9.9934336],
"Grünheide": [52.4262298, 13.8226458],
"Leiden": [52.1594747, 4.4908843],
"Boadilla del Monte": [40.4053485, -3.8764663],
"Bracknell": [51.4143513, -0.7449925],
"Venlo": [51.39244885, 6.1511724144122955],
"Münster": [51.9625101, 7.6251879],
"Orosháza": [46.5603339, 20.6693477],
"Osnabrück": [52.2719595, 8.047635],
"Vitry-sur-Seine": [48.7876, 2.39164],
"Mönchengladbach": [51.1947131, 6.4353792],
"Carpiano": [45.3405722, 9.2727326],
"Rosate": [45.3519922, 9.0174784],
"Gaydon": [52.183882249999996, -1.4665136858172174],
"Pordenone": [45.9562503, 12.6597197],
"Kosice": [48.7172272, 21.2496774],
"Charenton-le-Pont": [48.8198479, 2.4159508],
"Liège": [50.6450944, 5.5736112],
"Raamsdonksveer": [51.6961345, 4.8745894],
"Matera": [40.447641899999994, 16.47357384028852],
"Jonkoping County": [57.535426, 14.641181601671247],
"Croix": [50.6802124, 3.1564365],
"Helsinki": [60.1674881, 24.9427473],
"Luleå": [65.5831187, 22.1459535],
"Genoa": [44.40726, 8.9338624],
"Bernay": [49.0902278, 0.5989157],
"Helsingborg": [56.0442098, 12.703706],
"Fontenay-sous-Bois": [48.8490721, 2.4749347],
"Vila do Conde": [41.3540398, -8.7435403],
"Mainz": [50.0012314, 8.2762513],
"Bollate": [45.544479, 9.1151877],
"Pieve Emanuele": [45.3559126, 9.2020722],
"Krakow": [50.0619474, 19.9368564],
"Guyancourt": [48.7709414, 2.0706084],
"Sant Just Desvern": [41.3815832, 2.0750677],
"Nice": [43.7009358, 7.2683912],
"San Colombano al Lambro": [45.1817156, 9.488578],
"Dublin City": [53.3498006, -6.2602964],
"Basingstoke": [51.262826, -1.0861976],
"Sundbyberg": [59.3628802, 17.9681359],
"Verona": [45.4384958, 10.9924122],
"Albuzzano": [45.1866257, 9.27504],
"Arconate": [45.5406637, 8.8481616],
"Birkirkara": [35.8994547, 14.4649578],
"St-Malo": [48.649518, -2.0260409],
"Sligo": [54.2720696, -8.4751357],
"Crolles": [45.2845907, 5.8839432],
"Trezzano Rosa": [45.5828314, 9.4870409],
"Plaisir": [48.817399, 1.9476363],
"Imola": [44.3535145, 11.7141233],
"Bègles": [44.8074559, -0.5488542],
"Novate Milanese": [45.5319497, 9.1401207],
"Waddinxveen": [52.0443437, 4.641596120380745],
"Palaiseau": [48.7144587, 2.2453842],
"Szentendre": [47.6677606, 19.0760467],
"Moravia-Silesia": [49.860275, 18.038654134727693],
"Cassano d'Adda": [45.5263333, 9.5214907],
"Puteaux": [48.8841522, 2.2368863],
"Székesfehérvár": [47.1910169, 18.410811],
"Cork": [51.897077, -8.4654674],
"Granarolo dell'Emilia": [44.55321, 11.4432081],
"Malmö": [55.6052931, 13.0001566],
"Sedriano": [45.4883326, 8.9687313],
"Nitra Region": [48.31295, 18.0894593],
"Paterna": [39.5039227, -0.4421014],
"Ancona": [43.480120400000004, 13.218790609151764],
"Vanzaghello": [45.5792526, 8.7823717],
"Recanati": [43.4035085, 13.5486016],
"Trezzo sull'Adda": [45.6087178, 9.5198437],
"Getafe": [40.3081807, -3.7302679],
"Arcole": [45.3582108, 11.2860916],
"Ostellato": [44.745678, 11.943521],
"Belas": [-9.1033375, 13.174545441548965],
"Koerich": [49.6697542, 5.949443],
"Sant Cugat del Vallès": [41.4728432, 2.0817809],
"Marcallo con Casone": [45.4872199, 8.87327605576723],
"Palma": [39.5695818, 2.6500745],
"Valencia": [39.4697065, -0.3763353],
"Vermezzo": [45.394647, 8.9810625],
"Palaión Fáliron": [37.9253691, 23.7006539]
}

for city in cities:
  if city in coordinates:
    continue
  geo = geocode(city)
  if not geo:
    print('ERROR for ', city)
    continue
  coordinates[city] = [geo.latitude, geo.longitude]
  sleep(1)
  print(city, coordinates[city])

In [8]:
final_data.query('city == "\'s-Hertogenbosch"')[['city','location']]

,city,location
2531,'s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2532,'s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2533,'s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2534,'s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2535,'s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"


In [9]:
# raw_data_0306.loc[raw_data_0306['location_temp'] == 'Iasi ', 'country'] = "Romania"
final_data.loc[final_data['city'] == "\'s-Hertogenbosch", 'city'] = "s-Hertogenbosch"

In [10]:
final_data.query('city == "s-Hertogenbosch"')[['city','location']]

,city,location
2531,s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2532,s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2533,s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2534,s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"
2535,s-Hertogenbosch,"'s-Hertogenbosch, North Brabant, Netherlands"


Объявив функцию для создания нового столбца с координатами

In [11]:
def coord(row):
  for item in coordinates:
    if item == row:
      return coordinates[item]
    if row == 'not_found':
      return 'not_found'

In [12]:
final_data['coord'] = final_data['city'].apply(coord)

In [14]:
final_data[['coord', 'city']]

,coord,city
0,"[47.5581077, 7.5878261]",Basel
1,"[47.5581077, 7.5878261]",Basel
2,"[47.5581077, 7.5878261]",Basel
3,"[47.5581077, 7.5878261]",Basel
4,"[47.5581077, 7.5878261]",Basel
...,...,...
4711,"[45.4641943, 9.1896346]",Milan
4712,"[45.4641943, 9.1896346]",Milan
4713,"[45.4641943, 9.1896346]",Milan
4714,"[39.4697065, -0.3763353]",Valencia


In [15]:
final_data.to_csv (r'final_data_1306.csv', index= False)